# Using Google Geocoding with Premium Plan (Client ID and Crypto)

This notebook demos how to use Google's Geocoding API to get the geometry info (lat and lng) from address info, note the python code is in python 2.6

#### For using in python 3, replace urlparse package with urllib.parse and modified function as below:

import urllib.parse

def sign_url(input_url=None, secret=None):

  if not input_url or not secret:
  
     raise Exception("Both input_url and secret are required")

  url = urllib.parse.urlparse(input_url)
  
  url_to_sign = url.path + "?" + url.query
  
  decoded_key = base64.urlsafe_b64decode(secret)
  
  signature = hmac.new(decoded_key, url_to_sign.encode('utf-8'), hashlib.sha1)
  
  encoded_signature = base64.urlsafe_b64encode(signature.digest())

  original_url = url.scheme + "://" + url.netloc + url.path + "?" + url.query

  return original_url + "&signature=" + encoded_signature.decode('ascii')
  
#### The demo here process a csv file that is generated from existing table on Impala

In [1]:
import hashlib
import hmac
import base64
import urlparse
import requests

import pandas as pd
from pyspark.sql.functions import udf

In [2]:
url_suffix = "&client=gme-novantas"
base_url = "https://maps.googleapis.com/maps/api/geocode/json?address="
secret = "the crypto key associated with the premium google map API account"

In [3]:
def sign_url(input_url=None, secret=None):
  """ Sign a request URL with a URL signing secret.

      Usage:
      from urlsigner import sign_url

      signed_url = sign_url(input_url=my_url, secret=SECRET)

      Args:
      input_url - The URL to sign
      secret    - Your URL signing secret

      Returns:
      The signed request URL
  """

  if not input_url or not secret:
    raise Exception("Both input_url and secret are required")

  url = urlparse.urlparse(input_url)

  # We only need to sign the path+query part of the string
  url_to_sign = url.path + "?" + url.query

  # Decode the private key into its binary format
  # We need to decode the URL-encoded private key
  decoded_key = base64.urlsafe_b64decode(secret)

  # Create a signature using the private key and the URL-encoded
  # string using HMAC SHA1. This signature will be binary.
  signature = hmac.new(decoded_key, url_to_sign, hashlib.sha1)

  # Encode the binary signature into base64 for use within a URL
  encoded_signature = base64.urlsafe_b64encode(signature.digest())

  original_url = url.scheme + "://" + url.netloc + url.path + "?" + url.query

  # Return signed URL
  return original_url + "&signature=" + encoded_signature


In [4]:
def lat(signed_url):
    result = requests.get(signed_url).json()
    return result['results'][0]['geometry']['location']['lat']

def lng(signed_url):
    result = requests.get(signed_url).json()
    return result['results'][0]['geometry']['location']['lng']

#### The dataframe is generated from a csv that is the query result from an existing impala table with only address info and lack of geometry info

In [5]:
df = pd.read_csv('../Data/query-impala-81380.csv')

#### Set the dataframe to show no truncate to its columns

In [6]:
pd.set_option('display.max_colwidth', -1)

In [7]:
df

,address,city,province,postal_code
0,271 King St,Midland,ON,L4R 4K6
1,600 Queen's Plate Dr,Etobicoke,ON,M9W 6Z7
2,1 Place Ville Marie - 8th Flr,Montreal,QC,H3C 3A9
3,125 Goulet St,Winnipeg,MB,R2H OR6
4,430 Boul Arthur Sauve,St-Eustache,QC,J7R 6V6
5,152 South Alberni Hwy,Parksville,BC,V6P 2G5
6,917 Queenston Rd,Stoney Creek,ON,L8E 1H8
7,728 Northmount Dr NW - Transit 1659,Calgary,AB,T2K 2K2
8,452 Dundas St,Woodstock,ON,N4S 7X6
9,4261 Highway 7,Unionville,ON,L3R 1L5


#### Note the address input in Google Map API should not contain space, using '+' to replace any space in address info

In [8]:
def get_addr(row):
    return row['address'].replace(' ', '+') + '+' + row['city'].replace(' ', '+') + '+' + row['province'] + '+' + row['postal_code'].replace(' ', '+')

In [9]:
df['full_address'] =  df.apply(lambda row: get_addr(row), axis = 1)

In [10]:
def get_input_url(full_address):
    return base_url + full_address + url_suffix

In [11]:
df['input_url'] =  df.apply(lambda row: get_input_url(row['full_address']), axis = 1)

In [12]:
df['input_url']

0     https://maps.googleapis.com/maps/api/geocode/json?address=271+King+St+Midland+ON+L4R+4K6&client=gme-novantas                        
1     https://maps.googleapis.com/maps/api/geocode/json?address=600+Queen's+Plate+Dr+Etobicoke+ON+M9W+6Z7&client=gme-novantas             
2     https://maps.googleapis.com/maps/api/geocode/json?address=1+Place+Ville+Marie+-+8th+Flr+Montreal+QC+H3C+3A9&client=gme-novantas     
3     https://maps.googleapis.com/maps/api/geocode/json?address=125+Goulet+St+Winnipeg+MB+R2H+OR6&client=gme-novantas                     
4     https://maps.googleapis.com/maps/api/geocode/json?address=430+Boul+Arthur+Sauve+St-Eustache+QC+J7R+6V6&client=gme-novantas          
5     https://maps.googleapis.com/maps/api/geocode/json?address=152+South+Alberni+Hwy+Parksville+BC+V6P+2G5&client=gme-novantas           
6     https://maps.googleapis.com/maps/api/geocode/json?address=917+Queenston+Rd+Stoney+Creek+ON+L8E+1H8&client=gme-novantas              
7     https://maps.googleap

In [13]:
df['signed_url'] =  df.apply(lambda row: sign_url(str(row['input_url']), secret), axis = 1)

In [14]:
df['signed_url']

0     https://maps.googleapis.com/maps/api/geocode/json?address=271+King+St+Midland+ON+L4R+4K6&client=gme-novantas&signature=tO8Nn5heId4E92Egs3WwsJKeRgE=                        
1     https://maps.googleapis.com/maps/api/geocode/json?address=600+Queen's+Plate+Dr+Etobicoke+ON+M9W+6Z7&client=gme-novantas&signature=Hp9RHLD1OBTpRYiKYiFxwZmitTM=             
2     https://maps.googleapis.com/maps/api/geocode/json?address=1+Place+Ville+Marie+-+8th+Flr+Montreal+QC+H3C+3A9&client=gme-novantas&signature=0K1AXuUr17jVn-rslvsMI0rAOtw=     
3     https://maps.googleapis.com/maps/api/geocode/json?address=125+Goulet+St+Winnipeg+MB+R2H+OR6&client=gme-novantas&signature=593KUiwxI40LxNEgY_4NrM_GmEk=                     
4     https://maps.googleapis.com/maps/api/geocode/json?address=430+Boul+Arthur+Sauve+St-Eustache+QC+J7R+6V6&client=gme-novantas&signature=IoCq0tXHUmNYbP2gwC7e5dUnSpw=          
5     https://maps.googleapis.com/maps/api/geocode/json?address=152+South+Alberni+Hwy+Parksville+BC+V6P+2G5&cl

In [15]:
df['lat'] = df.apply(lambda row: lat(row['signed_url']), axis=1)

In [16]:
df['lng'] = df.apply(lambda row: lng(row['signed_url']), axis=1)

In [17]:
df[['address', 'city','province','postal_code','lat','lng']]

,address,city,province,postal_code,lat,lng
0,271 King St,Midland,ON,L4R 4K6,44.750329,-79.885287
1,600 Queen's Plate Dr,Etobicoke,ON,M9W 6Z7,43.723078,-79.598838
2,1 Place Ville Marie - 8th Flr,Montreal,QC,H3C 3A9,45.501476,-73.569253
3,125 Goulet St,Winnipeg,MB,R2H OR6,49.882658,-97.123260
4,430 Boul Arthur Sauve,St-Eustache,QC,J7R 6V6,45.565275,-73.906676
5,152 South Alberni Hwy,Parksville,BC,V6P 2G5,49.319199,-124.313387
6,917 Queenston Rd,Stoney Creek,ON,L8E 1H8,43.226339,-79.754270
7,728 Northmount Dr NW - Transit 1659,Calgary,AB,T2K 2K2,51.082540,-114.092944
8,452 Dundas St,Woodstock,ON,N4S 7X6,43.129666,-80.758118
9,4261 Highway 7,Unionville,ON,L3R 1L5,43.859076,-79.314603
